In [1]:
from google.colab import files
uploaded = files.upload()


Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [2]:
import pandas as pd

In [3]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
submission = pd.read_csv("sample_submission.csv")

In [4]:
test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [5]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [6]:
submission.head()

,id,sales
0,0,52
1,1,52
2,2,52
3,3,52
4,4,52


In [11]:
train.isnull().sum()

,0
date,0
store,0
item,0
sales,0


In [13]:
submission.isnull().sum()

,0
id,0
sales,0


In [16]:
print("test info is", test.info())
print("train info is", train.info())
print("submission info is", submission.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45000 non-null  int64 
 1   date    45000 non-null  object
 2   store   45000 non-null  int64 
 3   item    45000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ MB
test info is None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB
train info is None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      45000 non-null 

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# 1. Chargement des données
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 2. Feature Engineering (Extraction des caractéristiques de date)
def create_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    return df

train_df = create_features(train)
test_df = create_features(test)

# Définition des colonnes d'entrée (X) et de la cible (y)
FEATURES = ['store', 'item', 'year', 'month', 'day', 'dayofweek', 'quarter']
TARGET = 'sales'

# 3. Split Temporel (Validation sur les 3 derniers mois)
# On ne fait pas un split aléatoire car l'ordre des dates est crucial
train_df = train_df.sort_values('date')
split_date = '2017-10-01'  # On garde la fin de 2017 pour tester

df_train = train_df[train_df['date'] < split_date]
df_val = train_df[train_df['date'] >= split_date]

X_train, y_train = df_train[FEATURES], df_train[TARGET]
X_val, y_val = df_val[FEATURES], df_val[TARGET]

print(f"Taille Entraînement : {len(X_train)} lignes")
print(f"Taille Validation : {len(X_val)} lignes")

# 4. Entraînement du Random Forest
print("\n--- Entraînement Random Forest ---")
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_val)

# 5. Entraînement du XGBoost
print("--- Entraînement XGBoost ---")
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_val)

# 6. Évaluation et Comparaison
def get_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"[{model_name}] MAE: {mae:.4f} | RMSE: {rmse:.4f}")
    return mae, rmse

rf_mae, rf_rmse = get_metrics(y_val, rf_preds, "Random Forest")
xgb_mae, xgb_rmse = get_metrics(y_val, xgb_preds, "XGBoost")

# 7. Choix du meilleur modèle pour la prédiction finale
best_model = rf_model if rf_rmse < xgb_rmse else xgb_model
print(f"\nMeilleur modèle retenu : {'Random Forest' if best_model == rf_model else 'XGBoost'}")

# Prédiction sur le fichier test.csv
test_preds = best_model.predict(test_df[FEATURES])



Taille Entraînement : 867000 lignes
Taille Validation : 46000 lignes

--- Entraînement Random Forest ---
--- Entraînement XGBoost ---
[Random Forest] MAE: 14.0928 | RMSE: 19.0200
[XGBoost] MAE: 6.3401 | RMSE: 8.1899

Meilleur modèle retenu : XGBoost


In [20]:
# Création du fichier de soumission
submission = pd.DataFrame({'id': test['id'], 'sales': test_preds})
submission.to_csv('submission_local.csv', index=False)
print("\nFichier 'submission_local.csv' généré avec succès !")


Fichier 'submission_local.csv' généré avec succès !


In [21]:
submission = pd.read_csv("submission_local.csv")
submission.head()

,id,sales
0,0,11.671620
1,1,14.501775
2,2,14.527708
3,3,14.206195
4,4,14.951419
